In [8]:
import pandas as pd
import numpy as np
import pyrankvote
from pyrankvote import Candidate, Ballot
from IPython.display import Image

df = pd.read_csv("/Users/anxhela/Downloads/us_ca_sfo_2015_11_sheriff.normalized.csv")

In [9]:
df.head()

ballot_id  rank          choice
0      26105     1  VICKI HENNESSY
1      26105     2   JOHN ROBINSON
2      26105     3      $UNDERVOTE
3      26106     1  VICKI HENNESSY
4      26106     2   JOHN ROBINSON

In [10]:
df.choice.unique()

array(['VICKI HENNESSY', 'JOHN ROBINSON', '$UNDERVOTE', 'ROSS MIRKARIMI',
       '$OVERVOTE'], dtype=object)

In [41]:
df.shape

(609207, 3)

In [42]:
#we want to transform so that 

In [43]:
# Candidates:

hennessy = Candidate('VICKI HENNESSY')
robinson = Candidate('JOHN ROBINSON')
mirkarimi = Candidate('ROSS MIRKARIMI')

candidates = [hennessy, robinson, mirkarimi]

In [44]:
# df['rank'] = df['rank'].astype('str')
#df = df.groupby('ballot_id').apply(lambda x: list(np.unique(x)))

In [45]:
# Preprocessing Steps:

# df1 = df.pivot(index='ballot_id', columns='rank', values='choice').rename_axis(None, axis=1).reset_index()
# df1.head()

In [46]:
#Removing NaN with 0:

df1 = df1.replace(np.nan, '0')
df1.head()

ballot_id               1               2               3  \
4       26109  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
7       26112  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
8       26113  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
13      26118   JOHN ROBINSON  VICKI HENNESSY  ROSS MIRKARIMI   
17      26122  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   

                                ballot_choices_list  \
4   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
7   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
8   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
13  [JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI]   
17  [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   

         ballot_choices_list_with_candidate_objects  \
4   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
7   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
8   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
13  [JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI]   
17  [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   

                                       ballot_objects  
4   <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...  
7   <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...  
8   <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...  
13  <Ballot(JOHN ROBINSON, VICKI HENNESSY, ROSS MI...  
17  <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...

In [47]:
list_of_ballot_choices = df1[[1, 2, 3]].values.tolist()

In [48]:
for x in list_of_ballot_choices:
    while '0' in x:
        x.remove('0')

In [49]:
#Making a list of ballot choices:

df1['ballot_choices_list'] = list_of_ballot_choices
df1.head()

ballot_id               1               2               3  \
4       26109  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
7       26112  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
8       26113  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
13      26118   JOHN ROBINSON  VICKI HENNESSY  ROSS MIRKARIMI   
17      26122  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   

                                ballot_choices_list  \
4   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
7   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
8   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
13  [JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI]   
17  [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   

         ballot_choices_list_with_candidate_objects  \
4   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
7   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
8   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
13  [JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI]   
17  [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   

                                       ballot_objects  
4   <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...  
7   <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...  
8   <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...  
13  <Ballot(JOHN ROBINSON, VICKI HENNESSY, ROSS MI...  
17  <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...

In [50]:
#function to remove anomolies which are defined below
def remove_anomalies_from_ballots(ballots, value_to_remove):
    """
    ballots: series from dataframe. Each row contains a list of ranked candidates representing 1 ballot.
    value_to_remove: string that a ballot contains as a condition for its deletion
    """
    indices_to_drop = []
    
    for index, value in enumerate(ballots):
        if value_to_remove in value:
            indices_to_drop.append(index)
                   
    return indices_to_drop


In [51]:
df1.shape

(53191, 7)

In [52]:
# Removing write-ins, undervotes, and overvotes

write_in_indices_to_drop = remove_anomalies_from_ballots(df1['ballot_choices_list'], 'Write-in')
undervote_indices_to_drop = remove_anomalies_from_ballots(df1['ballot_choices_list'], '$UNDERVOTE')
overvote_indices_to_drop = remove_anomalies_from_ballots(df1['ballot_choices_list'], '$OVERVOTE')

indices_to_drop = write_in_indices_to_drop +  undervote_indices_to_drop + overvote_indices_to_drop

In [16]:
df1.head()

ballot_id               1              2               3  \
0      26105  VICKI HENNESSY  JOHN ROBINSON      $UNDERVOTE   
1      26106  VICKI HENNESSY  JOHN ROBINSON      $UNDERVOTE   
2      26107  VICKI HENNESSY     $UNDERVOTE      $UNDERVOTE   
3      26108  VICKI HENNESSY  JOHN ROBINSON      $UNDERVOTE   
4      26109  VICKI HENNESSY  JOHN ROBINSON  ROSS MIRKARIMI   

                               ballot_choices_list  
0      [VICKI HENNESSY, JOHN ROBINSON, $UNDERVOTE]  
1      [VICKI HENNESSY, JOHN ROBINSON, $UNDERVOTE]  
2         [VICKI HENNESSY, $UNDERVOTE, $UNDERVOTE]  
3      [VICKI HENNESSY, JOHN ROBINSON, $UNDERVOTE]  
4  [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]

In [17]:
#removing the anomolies defined above using function
df1 = df1.drop(index=indices_to_drop)
df1.shape

(53191, 5)

In [18]:
df1.head()

ballot_id               1               2               3  \
4       26109  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
7       26112  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
8       26113  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
13      26118   JOHN ROBINSON  VICKI HENNESSY  ROSS MIRKARIMI   
17      26122  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   

                                ballot_choices_list  
4   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]  
7   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]  
8   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]  
13  [JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI]  
17  [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]

In [19]:
# creating a ballot from ballot_choice_list

candidates_array = []

for candidate_list in df1['ballot_choices_list']:
    ballot = []
    for candidate in candidate_list:
        candidate = candidate.strip(' ')
        ballot.append(Candidate(candidate))
    candidates_array.append(ballot)
candidates_array

[[<Candidate('VICKI HENNESSY')>,
  <Candidate('JOHN ROBINSON')>,
  <Candidate('ROSS MIRKARIMI')>],
 [<Candidate('VICKI HENNESSY')>,
  <Candidate('JOHN ROBINSON')>,
  <Candidate('ROSS MIRKARIMI')>],
 [<Candidate('VICKI HENNESSY')>,
  <Candidate('JOHN ROBINSON')>,
  <Candidate('ROSS MIRKARIMI')>],
 [<Candidate('JOHN ROBINSON')>,
  <Candidate('VICKI HENNESSY')>,
  <Candidate('ROSS MIRKARIMI')>],
 [<Candidate('VICKI HENNESSY')>,
  <Candidate('JOHN ROBINSON')>,
  <Candidate('ROSS MIRKARIMI')>],
 [<Candidate('ROSS MIRKARIMI')>,
  <Candidate('VICKI HENNESSY')>,
  <Candidate('JOHN ROBINSON')>],
 [<Candidate('JOHN ROBINSON')>,
  <Candidate('ROSS MIRKARIMI')>,
  <Candidate('VICKI HENNESSY')>],
 [<Candidate('JOHN ROBINSON')>,
  <Candidate('VICKI HENNESSY')>,
  <Candidate('ROSS MIRKARIMI')>],
 [<Candidate('ROSS MIRKARIMI')>,
  <Candidate('VICKI HENNESSY')>,
  <Candidate('JOHN ROBINSON')>],
 [<Candidate('ROSS MIRKARIMI')>,
  <Candidate('VICKI HENNESSY')>,
  <Candidate('JOHN ROBINSON')>],
 [<Candida

In [22]:
#setting created ballot array as df column
df1['ballot_choices_list_with_candidate_objects'] = candidates_array

In [23]:
df1.head()

ballot_id               1               2               3  \
4       26109  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
7       26112  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
8       26113  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
13      26118   JOHN ROBINSON  VICKI HENNESSY  ROSS MIRKARIMI   
17      26122  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   

                                ballot_choices_list  \
4   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
7   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
8   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
13  [JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI]   
17  [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   

         ballot_choices_list_with_candidate_objects  
4   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]  
7   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]  
8   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]  
13  [JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI]  
17  [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]

In [24]:
#using pyrank's Ballot function to create ballots from the ballot choice object column

ballot_objects = []

for index,value in enumerate(df1['ballot_choices_list_with_candidate_objects']):
    ballot = Ballot(ranked_candidates=value)
    ballot_objects.append(ballot)
ballot_objects

[<Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI)>,
 <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI)>,
 <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI)>,
 <Ballot(JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI)>,
 <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI)>,
 <Ballot(ROSS MIRKARIMI, VICKI HENNESSY, JOHN ROBINSON)>,
 <Ballot(JOHN ROBINSON, ROSS MIRKARIMI, VICKI HENNESSY)>,
 <Ballot(JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI)>,
 <Ballot(ROSS MIRKARIMI, VICKI HENNESSY, JOHN ROBINSON)>,
 <Ballot(ROSS MIRKARIMI, VICKI HENNESSY, JOHN ROBINSON)>,
 <Ballot(VICKI HENNESSY, ROSS MIRKARIMI, JOHN ROBINSON)>,
 <Ballot(JOHN ROBINSON, ROSS MIRKARIMI, VICKI HENNESSY)>,
 <Ballot(JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI)>,
 <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI)>,
 <Ballot(ROSS MIRKARIMI, VICKI HENNESSY, JOHN ROBINSON)>,
 <Ballot(VICKI HENNESSY, ROSS MIRKARIMI, JOHN ROBINSON)>,
 <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI)>,
 <Ballot(ROSS 

In [25]:
df1['ballot_objects'] = ballot_objects
df1.head()

ballot_id               1               2               3  \
4       26109  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
7       26112  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
8       26113  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   
13      26118   JOHN ROBINSON  VICKI HENNESSY  ROSS MIRKARIMI   
17      26122  VICKI HENNESSY   JOHN ROBINSON  ROSS MIRKARIMI   

                                ballot_choices_list  \
4   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
7   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
8   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
13  [JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI]   
17  [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   

         ballot_choices_list_with_candidate_objects  \
4   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
7   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
8   [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   
13  [JOHN ROBINSON, VICKI HENNESSY, ROSS MIRKARIMI]   
17  [VICKI HENNESSY, JOHN ROBINSON, ROSS MIRKARIMI]   

                                       ballot_objects  
4   <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...  
7   <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...  
8   <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...  
13  <Ballot(JOHN ROBINSON, VICKI HENNESSY, ROSS MI...  
17  <Ballot(VICKI HENNESSY, JOHN ROBINSON, ROSS MI...

In [27]:
df.duplicated().sum()

0

In [26]:
#Run instant runoff 


In [29]:
election_result = pyrankvote.instant_runoff_voting(candidates, df1['ballot_objects'])
election_result

ElectionResults(3 rounds)

In [30]:
print(election_result)

ROUND 1
Candidate         Votes  Status
--------------  -------  --------
VICKI HENNESSY    28006  Hopeful
ROSS MIRKARIMI    20892  Hopeful
JOHN ROBINSON      4293  Hopeful

ROUND 2
Candidate         Votes  Status
--------------  -------  --------
VICKI HENNESSY    30829  Hopeful
ROSS MIRKARIMI    22362  Hopeful
JOHN ROBINSON         0  Rejected

FINAL RESULT
Candidate         Votes  Status
--------------  -------  --------
VICKI HENNESSY    53191  Elected
ROSS MIRKARIMI        0  Rejected
JOHN ROBINSON         0  Rejected



In [32]:
# IRV Winner:

winners = election_result.get_winners()
winners

[<Candidate('VICKI HENNESSY')>]

In [33]:
#Condorcet winner:

def create_ballot_dict(df1):
    all_ballots = list(df1['ballot_objects'].values)
    ballot_dict={}
    for i in range(len(all_ballots)):
        ballot_str=str(all_ballots[i])
        curr_ballot = all_ballots[i]
        if ballot_str in ballot_dict:
            ballot_dict[ballot_str][0]=ballot_dict[ballot_str][0]+1
        else:
            ballot_dict[ballot_str]=[1,all_ballots[i]]
    return ballot_dict

def create_candidate_matrix(candidates):
    candidate_names=[]
    for cand in candidates:
        candidate_names.append(cand.name)
        
    print("candidate_names: ",candidate_names)
    cand_matrix= pd.DataFrame(0, columns=candidate_names, index=candidate_names)
    print(cand_matrix)
    return cand_matrix

def symbolize_cand_matrix(cand_matrix):
    for i in cand_matrix.index:
        for j in cand_matrix.columns:
            if i ==j:
                cand_matrix.loc[i,j] ='`'
            else:
                if cand_matrix.loc[i,j]!= '++' and cand_matrix.loc[i,j]!= '--':
                    if cand_matrix.loc[i,j]>cand_matrix.loc[j,i]:
                        cand_matrix.loc[i,j] = '++'
                        cand_matrix.loc[j,i] = '--'
                    else:
                        cand_matrix.loc[i,j] = '--'
                        cand_matrix.loc[j,i] = '++'
    print(cand_matrix)

def countX(lst, x): 
    return lst.count(x) 

def return_winners(cand_matrix):

    result_dict={}
    for i in cand_matrix.index:
        key = ''.join([str(countX(list(cand_matrix.loc[i]),'--')+1), ". ",i])
        result_dict[key]= (countX(list(cand_matrix.loc[i]),'++'), countX(list(cand_matrix.loc[i]),'--'))
    for key in sorted(result_dict.keys()): 
        print("Rank %s: (Wins, losses) %s" % (key, result_dict[key]))

In [34]:
##Condorcet_winner returns the Condorcet results
##Input params: Dataframe with ballot_objects column and the list of candidates

def Condorcet_winner(df1, candidates):
    all_ballots = list(df1['ballot_objects'].values)
    print("Creating the ballot dictionary...")
    ballot_dict=create_ballot_dict(df1)
    print("Creating the candidate matrix to store pairwise results...")
    candidate_names=[]
    for cand in candidates:
        candidate_names.append(cand.name)
        
    print("\ncandidate_names: ",candidate_names)
    cand_matrix= pd.DataFrame(0, columns=candidate_names, index=candidate_names)
    print("\nUpdating the candidate matrix with pairwise results...")

    for ballot in ballot_dict:
        ranked_candidates = list(ballot_dict[ballot][1].ranked_candidates)
        for i in range(len(ranked_candidates)):
            for j in range(len(ranked_candidates)):
                if (ranked_candidates[i].name!=ranked_candidates[j].name) & (i<j):
                    cand_matrix[ranked_candidates[j].name][ranked_candidates[i].name] = cand_matrix[ranked_candidates[j].name][ranked_candidates[i].name]+ballot_dict[ballot][0]
    
    print("\n",cand_matrix)
    print("\nSymbolizing the candidate matrix with pairwise results...\n")

    symbolize_cand_matrix(cand_matrix)
    print("\n\nCondorcet results: ")
    return_winners(cand_matrix)

In [35]:
Condorcet_winner(df1, candidates)

Creating the ballot dictionary...
Creating the candidate matrix to store pairwise results...

candidate_names:  ['VICKI HENNESSY', 'JOHN ROBINSON', 'ROSS MIRKARIMI']

Updating the candidate matrix with pairwise results...

                 VICKI HENNESSY  JOHN ROBINSON  ROSS MIRKARIMI
VICKI HENNESSY               0          43620           30829
JOHN ROBINSON             9571              0           20607
ROSS MIRKARIMI           22362          32584               0

Symbolizing the candidate matrix with pairwise results...

               VICKI HENNESSY JOHN ROBINSON ROSS MIRKARIMI
VICKI HENNESSY              `            ++             ++
JOHN ROBINSON              --             `             --
ROSS MIRKARIMI             --            ++              `


Condorcet results: 
Rank 1. VICKI HENNESSY: (Wins, losses) (2, 0)
Rank 2. ROSS MIRKARIMI: (Wins, losses) (1, 1)
Rank 3. JOHN ROBINSON: (Wins, losses) (0, 2)


In [37]:
all_ballots = list(df1['ballot_objects'].values)
print("Creating the ballot dictionary...")
ballot_dict=create_ballot_dict(df1)
print("Creating the candidate matrix to store pairwise results...")
candidate_names=[]
for cand in candidates:
    candidate_names.append(cand.name)
    
print("\ncandidate_names: ",candidate_names)
cand_matrix= pd.DataFrame(0, columns=candidate_names, index=candidate_names)
print("\nUpdating the candidate matrix with pairwise results...")
for ballot in ballot_dict:
    ranked_candidates = list(ballot_dict[ballot][1].ranked_candidates)
    for i in range(len(ranked_candidates)):
        for j in range(len(ranked_candidates)):
            if (ranked_candidates[i].name!=ranked_candidates[j].name) & (i<j):
                cand_matrix[ranked_candidates[j].name][ranked_candidates[i].name] = cand_matrix[ranked_candidates[j].name][ranked_candidates[i].name]+ballot_dict[ballot][0]

print("\n",cand_matrix)

Creating the ballot dictionary...
Creating the candidate matrix to store pairwise results...

candidate_names:  ['VICKI HENNESSY', 'JOHN ROBINSON', 'ROSS MIRKARIMI']

Updating the candidate matrix with pairwise results...

                 VICKI HENNESSY  JOHN ROBINSON  ROSS MIRKARIMI
VICKI HENNESSY               0          43620           30829
JOHN ROBINSON             9571              0           20607
ROSS MIRKARIMI           22362          32584               0


In [38]:
cand_matrix1 = cand_matrix.copy(deep=True)
for i in cand_matrix1.columns:
    for j in cand_matrix1.index:
        cand_matrix1[i][j] = cand_matrix[i][j] - cand_matrix[j][i]
cand_matrix1

VICKI HENNESSY  JOHN ROBINSON  ROSS MIRKARIMI
VICKI HENNESSY               0          34049            8467
JOHN ROBINSON           -34049              0          -11977
ROSS MIRKARIMI           -8467          11977               0